In [95]:
import os
from dotenv import load_dotenv

load_dotenv() # load session

True

In [96]:
from aocd import get_data
day = 3
year = 2023

input = get_data(day=day, year=year)

In [97]:
def read_schematic(text):
  lines = text.splitlines()
  schematic = [[0 for _ in range(len(lines))] for _ in range(len(lines[0]))]
  for line_index, line in enumerate(lines):
    for column_index, char in enumerate(line):
      if char.isdigit():
        schematic[line_index][column_index] = { 'type': 'part_digit', 'value': char }
      elif char != '.':
        schematic[line_index][column_index] = { 'type': 'symbol', 'value': char }
      else:
        schematic[line_index][column_index] = { 'type': 'empty' }

  return schematic

In [117]:
example_input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

schematic = read_schematic(example_input)

In [118]:
def find_parts(schematic):
  parts = []
  for line_index, _ in enumerate(schematic):
    part_starting_column = None
    part_digits = []
    for column_index, column in enumerate(schematic[line_index]):
      if column['type'] == 'part_digit':
        part_starting_column = column_index if part_starting_column == None else part_starting_column
        part_digits.append(column['value'])

        if column_index == len(schematic[0]) - 1:
          parts.append((''.join(part_digits), line_index, part_starting_column))
          part_starting_column = None
          part_digits = []
      else:
        if part_starting_column != None:
          parts.append((''.join(part_digits), line_index, part_starting_column))
          part_starting_column = None
          part_digits = []

  return parts

find_parts(schematic)

[('467', 0, 0),
 ('114', 0, 5),
 ('35', 2, 2),
 ('633', 2, 6),
 ('617', 4, 0),
 ('58', 5, 7),
 ('592', 6, 2),
 ('755', 7, 6),
 ('664', 9, 1),
 ('598', 9, 5)]

In [119]:
def find_symbols(schematic):
    symbols = [(column['value'], line_index, column_index)
             for line_index, line in enumerate(schematic)
             for column_index, column in enumerate(line) if column['type'] == 'symbol']
    
    return symbols

find_symbols(schematic)

[('*', 1, 3), ('#', 3, 6), ('*', 4, 3), ('+', 5, 5), ('$', 8, 3), ('*', 8, 5)]

In [120]:
def adjacents_spaces_to(line_index, column_index, line_size, line_count):
  adjacents = [(line_index + l, column_index + c) 
               for l in [-1, 0, 1] if line_index + l >= 0 and line_index + l < line_count
               for c in [-1, 0, 1] if column_index + c >= 0 and column_index + c < line_size
               and (line_index + l, column_index + c) != (line_index, column_index)]
  return adjacents

adjacents_spaces_to(3, 1, 10, 10)

[(2, 0), (2, 1), (2, 2), (3, 0), (3, 2), (4, 0), (4, 1), (4, 2)]

In [121]:
parts = find_parts(schematic)

def is_part_adjacent_of_symbol(schematic, symbol, part):
  _, symbol_line, symbol_column = symbol
  part_number, part_line, part_column = part
  spaces_adjacents_to_symbol = adjacents_spaces_to(symbol_line, symbol_column, len(schematic), len(schematic[0]))
  for i in range(len(part_number)):
    if (part_line, part_column + i) in spaces_adjacents_to_symbol:
      return True
  return False

assert is_part_adjacent_of_symbol(schematic, ('*', 1, 3), ('467', 0, 0))
assert is_part_adjacent_of_symbol(schematic, ('*', 1, 3), ('114', 0, 5)) == False


In [122]:
def find_parts_adjacents_to_a_symbol(schematic):
  parts = find_parts(schematic)
  symbols = find_symbols(schematic)

  return [part
          for symbol in symbols
          for part in parts if is_part_adjacent_of_symbol(schematic, symbol, part)]

find_parts_adjacents_to_a_symbol(schematic)

assert sum(int(part[0]) for part in find_parts_adjacents_to_a_symbol(schematic)) == 4361


In [125]:
from aocd import submit

schematic = read_schematic(input)
parts = find_parts_adjacents_to_a_symbol(schematic)

result = sum(int(part[0]) for part in parts)

submit(answer=result, part='a', day=day, year=year)

That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


In [139]:
example_input = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

def find_gear_ratio(schematic):
  parts = find_parts(schematic)
  symbols = find_symbols(schematic)

  gear_ratios = []
  for symbol in symbols:
    if symbol[0] == '*':
      adjacent_parts = []
      for part in parts:
        if is_part_adjacent_of_symbol(schematic, symbol, part):
          adjacent_parts.append(part)
      if len(adjacent_parts) == 2:
        gear_ratios.append(int(adjacent_parts[0][0]) * int(adjacent_parts[1][0]))

  return sum(gear_ratios)

assert find_gear_ratio(read_schematic(example_input)) == 467835

In [143]:
schematic = read_schematic(input)

result = find_gear_ratio(schematic)

submit(answer=result, part='b', day=day, year=year)

That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 3! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
